In [ ]:
!pip install --upgrade kobart-transformers

In [ ]:
!pip install --upgrade transformers

In [ ]:
import json
import numpy as np
import pandas as pd

In [ ]:
import os

DIR = "/content/drive/MyDrive/DACON_LG/data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [ ]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [ ]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [ ]:
test['total'].head()

2000    제235회    본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군...
2002    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록...
2003    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결...
2004    제235회    본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분...
Name: total, dtype: object

In [ ]:
experiment_samples = test['total'].tolist()

In [ ]:
print(len(max(experiment_samples, key=len)))
print(len(min(experiment_samples, key=len)))

10983
96


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from kobart_transformers import get_kobart_for_conditional_generation

result_list = []
# model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
# model = get_kobart_model()
model = get_kobart_for_conditional_generation()
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
kobart_tokenizer = get_kobart_tokenizer()

for i in range(len(experiment_samples)):
  ARTICLE_TO_SUMMARIZE = experiment_samples[i]
  inputs = kobart_tokenizer([ARTICLE_TO_SUMMARIZE], max_length=10983, return_tensors='pt')

  # Generate Summary
  summary_ids = model.generate(inputs['input_ids'], num_beams=10, max_length=140, no_repeat_ngram_size=1, top_p=0.80, early_stopping=True)
  result = [kobart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  result_list.append(result)

In [ ]:
from kobart_transformers import get_kobart_model, get_kobart_tokenizer
from kobart_transformers import get_kobart_for_conditional_generation

kobart_tokenizer = get_kobart_tokenizer()
model = get_kobart_for_conditional_generation()

ARTICLE_TO_SUMMARIZE = experiment_samples[0]
inputs = kobart_tokenizer([ARTICLE_TO_SUMMARIZE], max_length=10983, return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'],
                             num_beams=10,
                             max_length=140,
                             no_repeat_ngram_size=1,
                             top_p=0.90,
                            #  num_return_sequences = 3,
                            #  top_k=50,
                             early_stopping=True)

print([kobart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


["- 제회 제235 회 ' 연차(2012.06.21.) 음성 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금 이제 음성 군이 있는 단, 이동에 진거(합원 대문 한자 동회의를 개의하겠습니다. 먼저 사제무과장으로부터 보고가 있으니까 있습니다. 거고정 김 저저와 함께 적 제가 있겠겠습니다. 구회사 무과장입니다. 따라서 지난 2012년 5월 29일 제134조 제주 경남 김해는 지방 제3 제4일 부산 및 본에서 서울을 조정 등 상 개정 고 외 결정 중에서 이송한 안건에 대해서 보고를 드리 합니다. 시작적으로 있던 것입니다. 그렇게 올해 5029 전 인터넷 북북 전북도 일", "- 제회 제235 회 ' 연차(2012.06.21.) 음성 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금 이제 음성 군이 있는 단, 이동에 진거(합원 대문 한자 동회의를 개의하겠습니다. 먼저 사제무과장으로부터 보고가 있으니까 있습니다. 거고정 김 저가지와 같이 적 제가 있겠겠습니다. 구회사 무과장입니다. 따라서 지난 2012년 5월 29일 제134조 제주 경남 김해는 지방 제3 제4일 부산 및 본에서 서울을 조정 등 상 개정 고 외 결정 중에서 이송한 안건에 대해서 보고를 드리 합니다. 시작적으로 있던 것입니다. 그렇게 올해 5029 전 인터넷 북북 전북도 일", "- 제회 제235 회 ' 연차(2012.06.21.) 음성 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금 이제 음성 군이 있는 단, 이동에 진거(합원 대문 한자 동회의를 개의하겠습니다. 먼저 사제무과장으로부터 보고가 있으니까 있습니다. 거고정 김 저저와 함께 적 제가 있겠겠습니다. 구회사 무과장입니다. 따라서 지난 2012년 5월 29일 제134조 제주 경남 김해는 지방 제3 제4일 부산 및 본에서 서울을 조정 등 상 개정 고 외 결정 중에서 이송한 안건에 대해서 보고를 드리 합니다. 시작적으로 있던 것입니다. 그렇게 올해 5029 전 인터넷 북북 전북산군"]


In [ ]:
result_list[:10]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON_LG/data/sample_submission.csv')

In [ ]:
submission['summary'] = result_list

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON_LG/data/kobart-trial.csv', index=False)